In [11]:
import polars as pl
import numpy as np
import pandas as pd

from mars.analysis.profiler import MarsDataProfiler

In [12]:

def get_demo_data():
    """生成一个包含各种‘脏’数据的 Demo 集，用于验证红绿灯报警逻辑"""
    np.random.seed(1234)
    rows = 10000
    
    # 1. 生成基础数据
    data = {
        "month": np.random.choice(["202401", "202402", "202403", "202404"], size=rows),
        "good_feature": np.random.normal(100, 10, size=rows),  # 完美特征
    }
    
    # 2. 注入脏数据 (测试 DQ 报警)
    
    # [Case A] 高缺失 (Missing Rate > 50%) -> show_missing 应全红
    bad_missing = np.random.normal(0, 1, size=rows)
    bad_missing[np.random.rand(rows) < 0.6] = -999 
    data["feat_high_missing"] = bad_missing
    
    # [Case B] 准常量 (Top1 Ratio > 99%) -> show_top1 应全红 [NEW]
    # 模拟：99.9% 的人都是 0，只有 0.1% 是 1
    quasi = np.zeros(rows)
    quasi[np.random.rand(rows) < 0.001] = 1
    data["feat_quasi_constant"] = quasi

    # [Case C] 高基数 (Unique Rate = 100%) -> show_unique 应全蓝 [NEW]
    # 模拟：用户 ID
    data["feat_user_id"] = [f"user_{i}" for i in range(rows)]

    # 3. 注入不稳定特征 (测试 Stats 报警)

    # [Case D] 剧烈抖动 (Stability/CV 爆炸) -> show_stats 应有红色长条
    # 模拟：前三个月均值100，第四个月均值突变成 1000
    unstable = np.random.normal(100, 5, size=rows)
    mask_apr = data["month"] == "202404"
    unstable[mask_apr] = unstable[mask_apr] * 10 
    data["feat_unstable"] = unstable

    return pl.DataFrame(data)

# 生成数据
df = get_demo_data()
print(f"✅ Demo Data Ready: {df.shape}")
print("包含测试场景: 正常, 高缺失, 准常量(Top1), 高基数(ID), 剧烈抖动(Unstable)")
df.head()

✅ Demo Data Ready: (10000, 6)
包含测试场景: 正常, 高缺失, 准常量(Top1), 高基数(ID), 剧烈抖动(Unstable)


month,good_feature,feat_high_missing,feat_quasi_constant,feat_user_id,feat_unstable
str,f64,f64,f64,str,f64
"""202404""",95.73776,-0.051205,0.0,"""user_0""",950.957766
"""202404""",109.295043,-999.0,0.0,"""user_1""",983.240373
"""202403""",104.267144,-0.809159,0.0,"""user_2""",101.073494
"""202402""",91.648176,0.277759,0.0,"""user_3""",103.256861
"""202401""",110.037528,1.419918,0.0,"""user_4""",98.321816


In [13]:
# 初始化分析器 (定义 -999 为缺失值)
profiler = MarsDataProfiler(df, custom_missing_values=[-999])

# 执行分组透视分析
report = profiler.generate_profile(profile_by="month") # 

[MARS] 2026-01-10 15:18:31 - INFO - Starting profiling (Profile by: month, Sparkline: True)...
[MARS] 2026-01-10 15:18:31 - INFO - Profile generated successfully.
[MARS] 2026-01-10 15:18:31 - INFO - ⏱️ [MarsDataProfiler.generate_profile] finished in 0.0272s


In [14]:
report

In [15]:
report.show_overview()

feature,dtype,distribution,missing_rate,zeros_rate,unique_rate,top1_ratio,mean,std,min,max,median,p25,p75
feat_high_missing,Float64,▂▂▃▆█▅▃▂,59.77%,0.00%,40.24%,59.77%,-597.09,489.91,-999.00,3.28,-999.00,-999.00,-0.28
feat_quasi_constant,Float64,█______▂,0.00%,99.88%,0.02%,99.88%,0.00,0.03,0.00,1.00,0.00,0.00,0.00
feat_unstable,Float64,█____▂▃▂,0.00%,0.00%,100.00%,0.01%,327.97,392.52,81.62,1166.73,102.13,97.88,887.05
good_feature,Float64,▂▂▄▇█▄▂▂,0.00%,0.00%,100.00%,0.01%,100.08,10.01,61.41,136.67,100.12,93.34,106.85
feat_user_id,String,None,0.00%,0.00%,100.00%,0.01%,nan,nan,nan,nan,nan,nan,nan
month,String,None,0.00%,0.00%,0.04%,25.31%,nan,nan,nan,nan,nan,nan,nan
